In [13]:
from keras.layers import Input,Dense,LeakyReLU,Conv2D,Conv2DTranspose,Reshape,Flatten, Dropout
from keras.models import Model

In [21]:
#Generator

latent_dim = 32
height = 32
width = 32
channels = 3
img_row_shape = 16
img_col_shape = 16
img_channel_num = 128 # 
channels = 3
generator_input = Input(shape=(latent_dim,))
x = Dense(img_channel_num * img_row_shape * img_col_shape)(generator_input)
x = LeakyReLU()(x)
x = Reshape((img_row_shape,img_col_shape,img_channel_num))(x)

x = Conv2D(256,5, padding= 'same')(x)
x = LeakyReLU()(x)

x = Conv2DTranspose(256,4, strides = 2, padding='same')(x)
x = LeakyReLU()(x)

x = Conv2D(256, 5, padding = 'same')(x)
x = LeakyReLU()(x)
x = Conv2D(256, 5, padding = 'same')(x)
x = LeakyReLU()(x)

x = Conv2D(channels, 7 , activation = 'tanh',padding = 'same')(x)
generator = Model(generator_input,x)

In [22]:
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_34 (LeakyReLU)   (None, 32768)             0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_35 (LeakyReLU)   (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

In [23]:
discriminator_input = Input(shape = (height,width,channels))
x = Conv2D(128,3)(discriminator_input)
x = LeakyReLU()(x)

x = Conv2D(128,4, strides = 2)(x)
x = LeakyReLU()(x)
x = Conv2D(128,4, strides = 2)(x)
x = LeakyReLU()(x)
x = Conv2D(128,4, strides = 2)(x)
x = LeakyReLU()(x)
x = Flatten()(x)

x = Dropout(0.4)(x)
x = Dense(1,activation = 'sigmoid')(x)
discriminator = Model(discriminator_input,x)
discriminator_optimizer = RMSprop(lr = 0.0008,clipvalue = 1.0, decay = 1e-8)
discriminator.compile(optimizer = discriminator_optimizer, loss = 'binary_crossentropy')

In [24]:
from keras.optimizers import RMSprop

In [25]:
discriminator.trainable = False

In [26]:
gan_input = Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input)) #ganerator
gan = Model(gan_input, gan_output)

In [ ]:
gan_optimizer = RMSprop(lr)